## Usefuls sources of informations:

* https://github.com/hbjornoy/deep-learning/blob/master/project2/helpers.py
* https://github.com/Ertugrulmert/EPFL-Deep-Learning-Projects/blob/master/Project_2/framework.py
* https://github.com/marieanselmet/DeepLearningEPFL_projects/tree/main/DL_framework_from_scratch

La grande différence est que tout le monde définit module avec un argument, et non une liste * -> à voir si on doit changer pour faire comme les années précédentes ou s'il y a qqc de plus pratique

In [181]:
import torch
import math
import time
import datetime
import random

from torch import Tensor
torch.set_grad_enabled(False)

In [2]:
# The father of all Modules
class Module(object):
    def forward(self, *input):
        raise NotImplementedError
        
    def backward(self, *gradwrtoutput):
        raise NotImplementedError
        
    def param(self):
        return []

## Activation

À part ReLU et Tanh c'est pas précisé dans l'énoncé - là c'est juste pour le flex

In [93]:
class ReLU(Module):
    def __init__(self):
        super().__init__()
        self.t = 0 
        
    def forward(self, input):
        self.t = input
        return input.clamp(0)

    def backward(self, gradwrtoutput):
        input = self.t
        sign = input.sign().clamp(0)
        return sign * gradwrtoutput
        
    def param(self):
        return []

In [94]:
class Tanh(Module):
    def __init__(self):
        super().__init__()
        self.t = 0 
        
    def forward(self, input):
        self.t = input
        
        out = []
        for x in input :
            e = math.exp(-2 * x)
            val = (1 - e) / (1 + e)
            out.append(val)
        return torch.FloatTensor(out)

    def backward(self, gradwrtoutput):
        z = self.t
        e = torch.exp(-2 * z)
        d_tanh = 4 * e / (1 + e)**2
        return d_tanh * gradwrtoutput
        
    def param(self):
        return []

In [90]:
class Sigmoid(Module):
    def __init__(self):
        super().__init__()
        self.t = 0 
        
    def forward(self, input):
        self.t = input
        return input.mul(-1).exp().add(1).pow(-1)

    def backward(self, gradwrtoutput):
        sig = self.t.mul(-1).exp().add(1).pow(-1)
        return sig.mul(-1).add(1).mul(sig)
        
    def param(self):
        return []

In [96]:
r = ReLU()
th = Tanh()
s = Sigmoid()

t = torch.FloatTensor([-10, 0, 1, 3, 10])

print(r.forward(t))
print(r.backward(t))
print(th.forward(t))
print(th.backward(t))
print(s.forward(t))
print(s.backward(t))

tensor([ 0.,  0.,  1.,  3., 10.])
tensor([-0.,  0.,  1.,  3., 10.])
tensor([-1.0000,  0.0000,  0.7616,  0.9951,  1.0000])
tensor([-8.2446e-08,  0.0000e+00,  4.1997e-01,  2.9598e-02,  8.2446e-08])
tensor([4.5398e-05, 5.0000e-01, 7.3106e-01, 9.5257e-01, 9.9995e-01])
tensor([4.5396e-05, 2.5000e-01, 1.9661e-01, 4.5177e-02, 4.5417e-05])


In [ ]:
class LeakyReLU(Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, *input):
        raise NotImplementedError

    def backward(self, *gradwrtoutput):
        raise NotImplementedError
        
    def param(self):
        return []

In [ ]:
class ELU(Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, *input):
        raise NotImplementedError

    def backward(self, *gradwrtoutput):
        raise NotImplementedError
        
    def param(self):
        return []

In [ ]:
class CReLU(Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, *input):
        raise NotImplementedError

    def backward(self, *gradwrtoutput):
        raise NotImplementedError
        
    def param(self):
        return []

## Linear

In [171]:
class Linear(Module):
    def __init__(self, input_size, output_size, mean=0, std=1):
        super().__init__()
        self.W = torch.empty(output_size, input_size).normal_(mean, std)
        self.b = torch.empty(output_size).normal_(mean, std)
        self.t = 0
        self.dW = torch.zeros(output_size, input_size)
        self.db = torch.zeros(output_size)
    def forward(self, input):
        self.t = input
        return self.W.mv(input).add(self.b)

    def backward(self, gradwrtoutput):
        self.dW.add_(gradwrtoutput.view(1, -1).t().mm(self.t.view(1, -1)))
        self.db.add_(gradwrtoutput)
        return self.W.t().mv(gradwrtoutput)
        
        
    def param(self):
        return [(self.W, self.dW), (self.b, self.db)]

In [172]:
l = Linear(10, 10)
t = torch.empty(10).fill_(1)
l.forward(t)
l.backward(t)

tensor([ 1.3606, -4.9119,  2.2291, -3.7748, -4.0492,  1.6691, -1.0099,  1.2334,
        -1.3293, -2.8563])

In [192]:
class Dropout(Module):
    def __init__(self, p = 0.5):
        super().__init__()
        self.p = p
        
    def forward(self, input):
        for i in range(input.shape[0]):
            if random.random() < self.p : 
                input[i] = 0 
        return input
    
    def backward(self, gradwrtoutput):
        return gradwrtoutput
        
        
    def param(self):
        return []

In [193]:
l = Dropout(0.5)
t = torch.empty(10).fill_(1)
l.forward(t)

tensor([1., 0., 0., 1., 0., 1., 1., 1., 0., 1.])

## Sequential

Lui en input il prend une liste de modules et son fwd c'est de feed le fwd de chaque layer dans l'autre

In [173]:
class Sequential(Module):
    def __init__(self, module_list):
        super().__init__()
        self.modules = module_list
        
    def forward(self, input):
        x = input
        for module in self.modules :
            x = module.forward(x)
        return x 

    def backward(self, gradwrtoutput):
        x = gradwrtoutput
        for module in self.modules :
            x = module.backward(x)
        return x
        
    def param(self):
        return []

In [176]:
s = Sequential([Linear(10,10), Linear(10,5), Linear(5,2)])


AttributeError: 'int' object has no attribute 'view'

## Loss

Seule lossMSE est demandé

In [ ]:
class LossMSE(Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, *input):
        raise NotImplementedError

    def backward(self, *gradwrtoutput):
        raise NotImplementedError
        
    def param(self):
        return []

In [ ]:
class LossMAE(Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, *input):
        raise NotImplementedError

    def backward(self, *gradwrtoutput):
        raise NotImplementedError
        
    def param(self):
        return []

In [ ]:
# Binary cross entropy loss

class LossBCE(Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, *input):
        raise NotImplementedError

    def backward(self, *gradwrtoutput):
        raise NotImplementedError
        
    def param(self):
        return []

## Optimizer

Pas demandé mais fancy

In [ ]:
class SGD():
    def __init__(self, params, lr):

In [ ]:
# Mazel Tov si on y arrive
class Adam():

## Train / Test

In [ ]:
def train():
    print("EKIP")
    
def test():
    print(667)

In [ ]:
train()

In [ ]:
test()

## Sandbox

In [ ]:
print("Faisons de la merde ici")